## Recursive Least Square with Kafka and Spark streaming 

This notebook provides an example for estimating the coefficients of a linear model on streaming data coming from a Kafka producer. The coefficient estimation is achieved using the recursive least square (RLS) algorithm, using two different RLS models in parallel (with different forgetting factors).

The linear model has 10 parameters, with coefficients [1,0,0,0,0,0,0,0,0,1] (see notebook KafkaSendRLS).

This notebook uses 
* the [Python client for the Apache Kafka distributed stream processing system](http://kafka-python.readthedocs.io/en/master/index.html) to receive messages from a Kafka cluster. 
* [Spark streaming](https://spark.apache.org/docs/latest/streaming-programming-guide.html) for processing the streaming data



### General import

In [28]:
import time
import re, ast
import numpy as np
import os
import findspark
findspark.init()

### Start Spark session


In [29]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

os.environ['PYSPARK_SUBMIT_ARGS'] = '--master local[*] pyspark-shell'

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("KafkaReceive") \
    .getOrCreate()

### Connect to Kafka cluster on topic dataLinearModel

In [30]:
#This function creates a connection to a Kafka stream
#You may change the topic, or batch interval
#The Zookeeper server is assumed to be running at 127.0.0.1:2181
#The function returns the Spark context, Spark streaming context, and DStream object
def getKafkaDStream(spark,topic='persistence',batch_interval=10):

    #Get Spark context
    sc=spark.sparkContext

    #Create streaming context, with required batch interval
    ssc = StreamingContext(sc, batch_interval)

    #Checkpointing needed for stateful transforms
    ssc.checkpoint("checkpoint")
    
    #Create a DStream that represents streaming data from Kafka, for the required topic 
    dstream = KafkaUtils.createStream(ssc, "zoo1:2181,zoo2:2181,zoo3:2181", "spark-streaming-consumer", {topic: 1})
    
    return [sc,ssc,dstream]

In [31]:
#Save state in global Python variable
def saveState(rdd):
    global state_global
    if rdd is not None:
        data=rdd.collect()
        state_global=data

In [32]:
def updateFunction(new_values, state): 
    ## RLS update function
    ## Only update with first value of RDD. You should transofrm new_values to array, and update models for all values 
    if (len(new_values)>0 ):
        
        key=new_values[0][0]
        yx=new_values[0][1]
        i=yx[0]
        y=yx[1]
        x=yx[2:]
        n=len(x)
        
        beta=state[1]
        beta.shape=(n,1)
        V=state[2]
        mu=state[3]
        sse=state[4]  ## sum of squared errors
        N=state[5]   ## number of treated samples
        x.shape=(1,n)
        err=y-x.dot(beta)
        sse=sse+pow(err,2.0)
        V=1.0/mu*(V-V.dot(x.T).dot(x).dot(V)/(1.0+float(x.dot(V).dot(x.T))))
        gamma=V.dot(x.T)
        beta=beta+gamma*err
        
        return (key,beta,V,mu,sse/(N+1.0),N+1)  ## update formula mod1
        
    else:
        return state

### Define streaming pipeline

* We define a stream with two states, for updating two RLS models in paralell. Each state contains a state of variables to keep the state of the model, as well as to keep track of MSE estimates. A state is a list of 5 elements:
    * The first three are beta, V and mu, and define the state of the model (see RLS formulas in course)
    * The last two are an estimate of the MSE of the model, and the number of treated samples 
* We create a DStream, flat map with the sensor ID as key, update state for the stream, and save MSE

In [33]:
import re, ast

n=24 # number of features
state_global = None

beta1=np.zeros(n)  ## initial parameter vector for model 1
V1=np.diag(np.zeros(n)+10) ## initial covariance matrix for model 1
mu1=1.0 # forgetting factor for model 1

state1=('mod1',beta1,V1,mu1,0,0)

beta2=np.zeros(n)  ## initial parameter vector for model 2
V2=np.diag(np.zeros(n)+1) ## initial covariance matrix for model 2
mu2=0.99 # forgetting factor for model 2

state2=('mod2',beta2,V2,mu2,0,0)

[sc,ssc,dstream]=getKafkaDStream(spark=spark,topic='RLSTrain',batch_interval=1)

#Evaluate input (a list - see KafkaSendRLS notebook)
dstream = dstream.map(lambda x: np.array(ast.literal_eval(x[1])))
#dstream.pprint()

dstream=dstream.flatMap(lambda x: [('mod1',('mod1',1.0*np.array(x))),
                            ('mod2',('mod2',1.0*np.array(x)))])
#dstream.pprint()

initialStateRDD = sc.parallelize([(u'mod1', state1),
                                  (u'mod2', state2)])

dstream=dstream.updateStateByKey(updateFunction,initialRDD=initialStateRDD)

#Only display beta and error
#beta should converge to [1,0,0,0,0,0,0,0,0,1] (send KafkaSend notebook)
theta = dstream.map(lambda x: x[1][0:2]+x[1][4:6])
theta.foreachRDD(saveState)
theta.pprint()





### Start streaming application

In [34]:
ssc.start()

-------------------------------------------
Time: 2019-05-23 16:21:00
-------------------------------------------
('mod1', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0)
('mod2', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0)

-------------------------------------------
Time: 2019-05-23 16:21:01
-------------------------------------------
('mod1', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0)
('mod2', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0)

-------------------------------------------
Time: 2019-05-23 16:21:02
-------------------------------------------
('mod1', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  

-------------------------------------------
Time: 2019-05-23 16:21:12
-------------------------------------------
('mod1', array([[ 0.04154363],
       [ 0.0421683 ],
       [ 0.0385064 ],
       [ 0.03364862],
       [ 0.00874991],
       [ 0.01686511],
       [ 0.05609032],
       [-0.01046118],
       [ 0.02564617],
       [ 0.01600392],
       [ 0.03785433],
       [ 0.04933562],
       [ 0.05119341],
       [ 0.04547034],
       [ 0.04739658],
       [ 0.06142461],
       [ 0.04325326],
       [ 0.04944136],
       [ 0.04251428],
       [ 0.04052692],
       [ 0.05072917],
       [ 0.03034966],
       [ 0.04508682],
       [ 0.04090152]]), array([[ 2.81232926]]), 5)
('mod2', array([[ 0.04149089],
       [ 0.04593909],
       [ 0.04127809],
       [ 0.0369167 ],
       [ 0.01955267],
       [ 0.02264688],
       [ 0.05704197],
       [-0.00180198],
       [-0.01429244],
       [ 0.01652074],
       [ 0.01762863],
       [ 0.05027121],
       [ 0.04699559],
       [ 0.04636927],
   

-------------------------------------------
Time: 2019-05-23 16:21:19
-------------------------------------------
('mod1', array([[-0.00487283],
       [ 0.72266049],
       [ 0.94426493],
       [-0.6396718 ],
       [ 0.29616956],
       [-0.10766827],
       [ 0.08704691],
       [ 0.01848252],
       [ 0.062691  ],
       [-0.2655163 ],
       [ 0.118293  ],
       [ 0.1637525 ],
       [-0.18072444],
       [ 0.02798378],
       [-0.05743179],
       [-0.03700452],
       [ 0.01434455],
       [-0.26972734],
       [ 0.12073799],
       [ 0.03504021],
       [ 0.3581541 ],
       [ 0.10266091],
       [-0.17471926],
       [-0.32463568]]), array([[ 0.13008182]]), 12)
('mod2', array([[ 0.03578722],
       [ 0.4782291 ],
       [ 0.69270008],
       [-0.2077947 ],
       [ 0.08613061],
       [-0.10389404],
       [ 0.09680527],
       [ 0.06530896],
       [-0.02317666],
       [-0.16570666],
       [-0.0171877 ],
       [ 0.11035678],
       [-0.0679619 ],
       [-0.00108377],
  

-------------------------------------------
Time: 2019-05-23 16:21:26
-------------------------------------------
('mod1', array([[-0.28572863],
       [ 0.70541184],
       [ 0.98588579],
       [-0.63092044],
       [ 0.6225325 ],
       [-0.28378575],
       [ 0.03808649],
       [-0.16543244],
       [-0.0796848 ],
       [-0.16629705],
       [ 0.21354124],
       [ 0.54172336],
       [ 0.26422427],
       [-1.05964958],
       [-0.16805174],
       [-0.36935713],
       [-0.18243486],
       [ 0.18109575],
       [ 0.34576383],
       [ 0.52122323],
       [ 0.30111215],
       [ 0.12457874],
       [-0.12290038],
       [-0.27806775]]), array([[ 0.76733462]]), 19)
('mod2', array([[-0.14100219],
       [ 0.43790495],
       [ 0.69824669],
       [-0.14413884],
       [ 0.24517993],
       [-0.18139057],
       [ 0.08099024],
       [-0.11589558],
       [-0.11223718],
       [-0.08296201],
       [ 0.08956685],
       [ 0.4181857 ],
       [ 0.31962826],
       [-0.87517832],
  

-------------------------------------------
Time: 2019-05-23 16:21:33
-------------------------------------------
('mod1', array([[ 0.7300287 ],
       [-1.72795757],
       [ 2.38770576],
       [-1.55155188],
       [ 1.21593859],
       [-0.86551413],
       [ 0.23460708],
       [-0.21055223],
       [ 0.27030307],
       [ 0.06127502],
       [-0.0510573 ],
       [-0.34672656],
       [ 0.98810054],
       [-1.15698902],
       [ 0.11667215],
       [ 0.18263984],
       [-0.05669887],
       [ 0.1800068 ],
       [-0.17882459],
       [ 0.34996802],
       [-0.35550006],
       [ 0.32738604],
       [-0.66385279],
       [ 1.14194384]]), array([[ 0.35229903]]), 26)
('mod2', array([[  4.46488008e-02],
       [ -1.13464654e-01],
       [  7.72849621e-01],
       [ -2.21462629e-01],
       [  3.95186313e-01],
       [ -5.43699669e-01],
       [ -9.05447616e-04],
       [ -5.61780613e-02],
       [  2.83549885e-01],
       [  3.08121328e-02],
       [ -1.11769510e-01],
       [  1.6

...

-------------------------------------------
Time: 2019-05-23 16:23:45
-------------------------------------------
('mod1', array([[  8.60216136e-02],
       [  8.83809763e-02],
       [  2.34513205e-01],
       [ -1.62269389e-01],
       [  6.43901419e-04],
       [  4.68175732e-02],
       [  9.71180243e-02],
       [ -1.35547486e-01],
       [  9.39617968e-02],
       [ -5.96575725e-02],
       [  9.67255136e-02],
       [ -1.63931409e-01],
       [  1.46460728e-01],
       [ -6.91941013e-02],
       [ -2.12098838e-02],
       [  7.28599518e-02],
       [ -1.18652255e-02],
       [ -5.68860324e-02],
       [ -4.72003998e-02],
       [  1.24597093e-01],
       [ -7.46920372e-02],
       [  1.09806568e-02],
       [ -3.20562520e-01],
       [  1.03093240e+00]]), array([[ 0.00140866]]), 158)
('mod2', array([[  9.30685127e-02],
       [  6.18091652e-02],
       [  2.24142777e-01],
       [ -1.39337731e-01],
       [ -8.46613136e-03],
       [  4.74067534e-02],
       [  1.00705872e-01],


-------------------------------------------
Time: 2019-05-23 16:23:51
-------------------------------------------
('mod1', array([[  8.42365298e-02],
       [  8.94308128e-02],
       [  2.32860107e-01],
       [ -1.63062909e-01],
       [  6.78250097e-04],
       [  4.67618391e-02],
       [  9.77710722e-02],
       [ -1.36101907e-01],
       [  9.41092352e-02],
       [ -5.88724972e-02],
       [  9.70761483e-02],
       [ -1.64215998e-01],
       [  1.46815008e-01],
       [ -6.89306896e-02],
       [ -2.15941564e-02],
       [  6.72469394e-02],
       [ -1.24929166e-02],
       [ -5.44710632e-02],
       [ -4.65685811e-02],
       [  1.25844340e-01],
       [ -7.41777559e-02],
       [  1.39310115e-02],
       [ -3.21675312e-01],
       [  1.03260269e+00]]), array([[  3.37545868e-06]]), 163)
('mod2', array([[  9.04937318e-02],
       [  6.42407434e-02],
       [  2.21477949e-01],
       [ -1.41194216e-01],
       [ -7.08376745e-03],
       [  4.71664431e-02],
       [  1.01626792e-

-------------------------------------------
Time: 2019-05-23 16:23:57
-------------------------------------------
('mod1', array([[  7.98091268e-02],
       [  9.04091859e-02],
       [  2.33605045e-01],
       [ -1.62855626e-01],
       [  5.15332038e-04],
       [  4.71244948e-02],
       [  9.88183587e-02],
       [ -1.35457480e-01],
       [  9.11388764e-02],
       [ -6.16636139e-02],
       [  9.71570659e-02],
       [ -1.65919639e-01],
       [  1.46905125e-01],
       [ -6.81475570e-02],
       [ -1.84058296e-02],
       [  6.88581851e-02],
       [ -1.21416592e-02],
       [ -5.38908386e-02],
       [ -4.51687199e-02],
       [  1.25497161e-01],
       [ -7.47486416e-02],
       [  1.36205075e-02],
       [ -3.23321044e-01],
       [  1.03582812e+00]]), array([[ 0.00080816]]), 168)
('mod2', array([[  8.39509925e-02],
       [  6.55511627e-02],
       [  2.22390536e-01],
       [ -1.40946857e-01],
       [ -7.77224138e-03],
       [  4.78364707e-02],
       [  1.03532868e-01],


-------------------------------------------
Time: 2019-05-23 16:24:03
-------------------------------------------
('mod1', array([[  7.98091268e-02],
       [  9.04091859e-02],
       [  2.33605045e-01],
       [ -1.62855626e-01],
       [  5.15332038e-04],
       [  4.71244948e-02],
       [  9.88183587e-02],
       [ -1.35457480e-01],
       [  9.11388764e-02],
       [ -6.16636139e-02],
       [  9.71570659e-02],
       [ -1.65919639e-01],
       [  1.46905125e-01],
       [ -6.81475570e-02],
       [ -1.84058296e-02],
       [  6.88581851e-02],
       [ -1.21416592e-02],
       [ -5.38908386e-02],
       [ -4.51687199e-02],
       [  1.25497161e-01],
       [ -7.47486416e-02],
       [  1.36205075e-02],
       [ -3.23321044e-01],
       [  1.03582812e+00]]), array([[ 0.00080816]]), 168)
('mod2', array([[  8.39509925e-02],
       [  6.55511627e-02],
       [  2.22390536e-01],
       [ -1.40946857e-01],
       [ -7.77224138e-03],
       [  4.78364707e-02],
       [  1.03532868e-01],


-------------------------------------------
Time: 2019-05-23 16:24:09
-------------------------------------------
('mod1', array([[  7.98091268e-02],
       [  9.04091859e-02],
       [  2.33605045e-01],
       [ -1.62855626e-01],
       [  5.15332038e-04],
       [  4.71244948e-02],
       [  9.88183587e-02],
       [ -1.35457480e-01],
       [  9.11388764e-02],
       [ -6.16636139e-02],
       [  9.71570659e-02],
       [ -1.65919639e-01],
       [  1.46905125e-01],
       [ -6.81475570e-02],
       [ -1.84058296e-02],
       [  6.88581851e-02],
       [ -1.21416592e-02],
       [ -5.38908386e-02],
       [ -4.51687199e-02],
       [  1.25497161e-01],
       [ -7.47486416e-02],
       [  1.36205075e-02],
       [ -3.23321044e-01],
       [  1.03582812e+00]]), array([[ 0.00080816]]), 168)
('mod2', array([[  8.39509925e-02],
       [  6.55511627e-02],
       [  2.22390536e-01],
       [ -1.40946857e-01],
       [ -7.77224138e-03],
       [  4.78364707e-02],
       [  1.03532868e-01],


### Stop streaming

In [24]:
ssc.stop(stopSparkContext=False,stopGraceFully=False)

-------------------------------------------
Time: 2019-05-23 14:55:40
-------------------------------------------
('mod1', array([[  7.98091268e-02],
       [  9.04091859e-02],
       [  2.33605045e-01],
       [ -1.62855626e-01],
       [  5.15332038e-04],
       [  4.71244948e-02],
       [  9.88183587e-02],
       [ -1.35457480e-01],
       [  9.11388764e-02],
       [ -6.16636139e-02],
       [  9.71570659e-02],
       [ -1.65919639e-01],
       [  1.46905125e-01],
       [ -6.81475570e-02],
       [ -1.84058296e-02],
       [  6.88581851e-02],
       [ -1.21416592e-02],
       [ -5.38908386e-02],
       [ -4.51687199e-02],
       [  1.25497161e-01],
       [ -7.47486416e-02],
       [  1.36205075e-02],
       [ -3.23321044e-01],
       [  1.03582812e+00]]), array([[ 0.00080816]]), 168)
('mod2', array([[  8.39509925e-02],
       [  6.55511627e-02],
       [  2.22390536e-01],
       [ -1.40946857e-01],
       [ -7.77224138e-03],
       [  4.78364707e-02],
       [  1.03532868e-01],


In [36]:
# predict the 8th day
theta1 = global_state[0][1] # mod1
theta2 = global_state[1][1] # mod2

#[sc,ssc,dstream2]=getKafkaDStream(spark=spark,topic='RLSTest',batch_interval=1)
#Evaluate input (a list - see KafkaSendRLS notebook)
#dstream2 = dstream.map(lambda x: np.array(ast.literal_eval(x[1])))
#dstream2.pprint()


-------------------------------------------
Time: 2019-05-23 16:26:29
-------------------------------------------
('mod1', array([[  7.98091268e-02],
       [  9.04091859e-02],
       [  2.33605045e-01],
       [ -1.62855626e-01],
       [  5.15332038e-04],
       [  4.71244948e-02],
       [  9.88183587e-02],
       [ -1.35457480e-01],
       [  9.11388764e-02],
       [ -6.16636139e-02],
       [  9.71570659e-02],
       [ -1.65919639e-01],
       [  1.46905125e-01],
       [ -6.81475570e-02],
       [ -1.84058296e-02],
       [  6.88581851e-02],
       [ -1.21416592e-02],
       [ -5.38908386e-02],
       [ -4.51687199e-02],
       [  1.25497161e-01],
       [ -7.47486416e-02],
       [  1.36205075e-02],
       [ -3.23321044e-01],
       [  1.03582812e+00]]), array([[ 0.00080816]]), 168)
('mod2', array([[  8.39509925e-02],
       [  6.55511627e-02],
       [  2.22390536e-01],
       [ -1.40946857e-01],
       [ -7.77224138e-03],
       [  4.78364707e-02],
       [  1.03532868e-01],


NameError: name 'global_state' is not defined

-------------------------------------------
Time: 2019-05-23 16:26:30
-------------------------------------------
('mod1', array([[  7.98091268e-02],
       [  9.04091859e-02],
       [  2.33605045e-01],
       [ -1.62855626e-01],
       [  5.15332038e-04],
       [  4.71244948e-02],
       [  9.88183587e-02],
       [ -1.35457480e-01],
       [  9.11388764e-02],
       [ -6.16636139e-02],
       [  9.71570659e-02],
       [ -1.65919639e-01],
       [  1.46905125e-01],
       [ -6.81475570e-02],
       [ -1.84058296e-02],
       [  6.88581851e-02],
       [ -1.21416592e-02],
       [ -5.38908386e-02],
       [ -4.51687199e-02],
       [  1.25497161e-01],
       [ -7.47486416e-02],
       [  1.36205075e-02],
       [ -3.23321044e-01],
       [  1.03582812e+00]]), array([[ 0.00080816]]), 168)
('mod2', array([[  8.39509925e-02],
       [  6.55511627e-02],
       [  2.22390536e-01],
       [ -1.40946857e-01],
       [ -7.77224138e-03],
       [  4.78364707e-02],
       [  1.03532868e-01],


-------------------------------------------
Time: 2019-05-23 16:26:36
-------------------------------------------
('mod1', array([[  7.98091268e-02],
       [  9.04091859e-02],
       [  2.33605045e-01],
       [ -1.62855626e-01],
       [  5.15332038e-04],
       [  4.71244948e-02],
       [  9.88183587e-02],
       [ -1.35457480e-01],
       [  9.11388764e-02],
       [ -6.16636139e-02],
       [  9.71570659e-02],
       [ -1.65919639e-01],
       [  1.46905125e-01],
       [ -6.81475570e-02],
       [ -1.84058296e-02],
       [  6.88581851e-02],
       [ -1.21416592e-02],
       [ -5.38908386e-02],
       [ -4.51687199e-02],
       [  1.25497161e-01],
       [ -7.47486416e-02],
       [  1.36205075e-02],
       [ -3.23321044e-01],
       [  1.03582812e+00]]), array([[ 0.00080816]]), 168)
('mod2', array([[  8.39509925e-02],
       [  6.55511627e-02],
       [  2.22390536e-01],
       [ -1.40946857e-01],
       [ -7.77224138e-03],
       [  4.78364707e-02],
       [  1.03532868e-01],
